# Collaborative Clustering

## Importing Libraries

In [1]:
import math
import numpy as np
import pandas as pd
# import pandas_datareader as web
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.linear_mode
#import LinearRegression
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras import Model
from keras.layers import Dense, LSTM, Dropout, Concatenate
from keras.optimizers import Adam
import matplotlib.pyplot as plt

#plt.style.use('fivethirtyeight')
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-v0_8-dark')

2024-05-04 00:38:03.589778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Importing Stocks dataframe

In [2]:
demo_df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AmazonStock_Sentiment_6M.csv');
demo_df

,date,open,low,high,volume,close,Name,Date,Positve,Negative,Neutral,Total,NeutralPos,NeutralNeg
0,10/2/2017,964.00,952.1201,967.305,2415846,959.19,AMZN,10/2/2017,37.088734,7.577268,55.333998,1003,37.088734,14.260219
1,10/3/2017,958.00,950.3700,963.690,2643484,957.10,AMZN,10/3/2017,38.159879,11.085973,50.754148,1326,38.159879,22.348774
2,10/4/2017,954.21,954.0500,967.790,2460721,965.45,AMZN,10/4/2017,36.168582,10.727969,53.103448,1305,36.168582,19.641470
3,10/5/2017,970.00,969.6400,981.510,3119487,980.85,AMZN,10/5/2017,47.939017,7.227555,44.833427,1771,47.939017,24.411076
4,10/6/2017,975.64,975.6400,995.750,3719840,989.58,AMZN,10/6/2017,34.628045,11.586570,53.785385,1519,34.628045,19.818134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,3/5/2018,1494.24,1481.0000,1525.380,5233934,1523.61,AMZN,3/5/2018,38.274182,7.794015,53.931802,1437,38.274182,15.879162
105,3/6/2018,1533.20,1528.0000,1542.130,4561718,1537.64,AMZN,3/6/2018,45.234604,6.818182,47.947214,1364,45.234604,20.887916
106,3/7/2018,1526.52,1522.5100,1545.900,4174123,1545.00,AMZN,3/7/2018,38.167260,9.786477,52.046263,1124,38.167260,19.757162
107,3/8/2018,1550.00,1545.2500,1554.880,3512528,1551.86,AMZN,3/8/2018,37.221728,8.726625,54.051647,1123,37.221728,16.691926


### Importing general user's decisions in the past

In [3]:
df_decisions = pd.read_csv('./stock_decisions.csv');
df_decisions.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Sector of stock,Buy/Sell/Keep,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg
0,0.016909,0.057341,0.000000,0.393651,0.209993,0.288915,0.354623,0.747695,0.353394,0.634112,0.363674,IT,Keep
1,0.006546,0.060510,0.003842,0.618285,0.282692,0.499607,0.399351,0.875622,0.132926,0.659470,0.270249,IT,Keep
2,0.000000,0.046401,0.030124,0.595365,0.277965,0.429087,0.316199,0.038742,0.334798,0.612328,0.318172,IT,Buy
3,0.027272,0.108571,0.042233,0.371262,0.382849,0.553326,0.807705,0.334854,0.926111,0.890981,0.149471,IT,Sell
4,0.037013,0.039837,0.058251,0.650335,0.326131,0.433689,0.251870,0.165370,0.620705,0.575857,0.332083,IT,Buy


In [4]:
df_decisions.drop('Sector of stock', axis=1, inplace=True)

### Taking Input from the user

In [5]:
input = []

todays_date = '2/5/2018'
list_of_stocks = []
list_of_stocks.append({'Percentage of portfolio': 2.5, 'Percentage of sector': 15.56, 'Sector of Stock': 'IT', 'comp': 'AMZN'});
list_of_stocks.append({'Percentage of portfolio': 4.5, 'Percentage of sector': 7.8, 'Sector of Stock': 'IT', 'comp': 'MSFT'});
list_of_stocks.append({'Percentage of portfolio': 2.5, 'Percentage of sector': 17.26, 'Sector of Stock': 'Health', 'comp': 'PFIZER'});
list_of_stocks

[{'Percentage of portfolio': 2.5,
  'Percentage of sector': 15.56,
  'Sector of Stock': 'IT',
  'comp': 'AMZN'},
 {'Percentage of portfolio': 4.5,
  'Percentage of sector': 7.8,
  'Sector of Stock': 'IT',
  'comp': 'MSFT'},
 {'Percentage of portfolio': 2.5,
  'Percentage of sector': 17.26,
  'Sector of Stock': 'Health',
  'comp': 'PFIZER'}]

In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [7]:
from tensorflow.keras.losses import MeanSquaredLogarithmicError, MeanSquaredError
from tensorflow.keras.saving import register_keras_serializable
@register_keras_serializable(package="MyLoss")
def MyLoss(y_true, y_pred):
    msle = MeanSquaredLogarithmicError()(y_true[:,0], y_pred[:,0])
    mse = MeanSquaredError()(y_true[:,1],y_pred[:,1])
    return  msle + mse

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
def makeNextDayVector(filtered_df, todays_date, stock):
    
    
    filtered_df['diff']=filtered_df['high']-filtered_df['low']
    filtered_df['exp']=(filtered_df['open']+filtered_df['close'])/2
    
       
    
    loaded_model = load_model('model.keras')
    curr_row = filtered_df[filtered_df['Date'] == todays_date]
    curr_row.drop(['date', 'Date', 'Name'], axis=1, inplace=True)
    filtered_df.drop(['date', 'Date', 'Name'], axis=1, inplace=True)
    curr_stock_value = curr_row['open'].iloc[0]
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    filtered_df[filtered_df.columns] = scaler.fit_transform(filtered_df[filtered_df.columns])
    
    predicted_values = pd.DataFrame()
    
    predicted_values[['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative','Neutral', 'Total', 'NeutralPos', 'NeutralNeg']] = 0
    
    predicted_values[['diff', 'exp']] = loaded_model.predict(np.array([filtered_df.values]).astype(float))
    # predicted_values=scaler.inverse_transform(predicted_values)
    print(predicted_values.columns)
    
    predicted_values[predicted_values.columns] = scaler.inverse_transform(predicted_values)
    
    
    predicted_values.drop(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative','Neutral', 'Total', 'NeutralPos', 'NeutralNeg'], axis=1, inplace=True)
    
    
    expected_value = predicted_values['exp'].iloc[0]
    error = predicted_values['diff'].iloc[0]
    
    percen_portfolio = stock['Percentage of portfolio']
    percen_sector = stock['Percentage of sector']
    sector = stock['Sector of Stock']
    
    positives = curr_row['Positve'].iloc[0]
    negatives = curr_row['Negative'].iloc[0]
    total = curr_row['Total'].iloc[0]
    neutral = curr_row['Neutral'].iloc[0]
    neutral_pos = curr_row['NeutralPos'].iloc[0]
    neutral_neg = curr_row['NeutralNeg'].iloc[0]
    
    
    
    stock_vect = []
    stock_vect.append(curr_stock_value)
    stock_vect.append(expected_value)
    stock_vect.append(error)
    stock_vect.append(percen_portfolio)
    stock_vect.append(percen_sector)
    stock_vect.append(positives)
    stock_vect.append(negatives)
    stock_vect.append(total)
    stock_vect.append(neutral)
    stock_vect.append(neutral_pos)
    stock_vect.append(neutral_neg)
    
    # add posi negi from filtered data in stock vector
    
    # stock_vect.append(sector)
    
    
    return stock_vect
    

In [9]:
import logging
logging.basicConfig(level=logging.INFO)

In [10]:
next_vectors = []
for i in range(len(list_of_stocks)):
    if(list_of_stocks[i]['comp'] == 'AMZN'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AmazonStock_Sentiment_6M.csv');
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'AMZN'})
    elif(list_of_stocks[i]['comp'] == 'AAPL'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AAPLStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'AAPL'})
    elif(list_of_stocks[i]['comp'] == 'MSFT'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/MSFTStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'MSFT'})
    elif(list_of_stocks[i]['comp'] == 'PFIZER'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/PfizerStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i]), 'comp': 'PFIZER'})


OSError: No file or directory found at model

In [ ]:
next_vectors[0]['next_day_vector']

[1402.62,
 1459.251255724728,
 61.63517479419708,
 2.5,
 15.56,
 36.95797198,
 12.6084056,
 1499,
 50.43362241,
 36.95797198,
 24.19173179]

In [ ]:
df_decisions.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Buy/Sell/Keep,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg
0,964.00,957.550,13.32,2.414993,5.338076,Buy,37.088734,7.577268,1003,55.333998,37.088734,14.260219
1,958.00,959.830,13.74,3.344313,19.608973,Buy,38.159879,11.085973,1326,50.754148,38.159879,22.348774
2,954.21,975.425,11.87,2.726208,13.568787,Buy,36.168582,10.727969,1305,53.103448,36.168582,19.641470
3,970.00,982.610,20.11,0.544740,9.648946,Buy,47.939017,7.227555,1771,44.833427,47.939017,24.411076
4,975.64,992.115,11.00,4.698582,6.444096,Buy,34.628045,11.586570,1519,53.785385,34.628045,19.818134


In [ ]:
scaler = MinMaxScaler()
columns_to_normalize = df_decisions.columns.difference(['Buy/Sell/Keep'])
normalized_values = scaler.fit_transform(df_decisions[columns_to_normalize])
normalized_df_decisions = pd.DataFrame(normalized_values, columns=columns_to_normalize)
normalized_df_decisions = pd.concat([normalized_df_decisions, df_decisions['Buy/Sell/Keep']], axis=1)
normalized_df_decisions
# normalized_df_decisions = pd.DataFrame(scaler.fit_transform(df_decisions), columns=columns_to_normalize)

,Current Value of Stock,Error in expected value of stock,Expected Value of Stock,Negative,Neutral,Neutral_Neg,Neutral_Pos,Percentage of portfolio,Percentage of sector,Positive,Total,Buy/Sell/Keep
0,0.622829,0.095228,0.611310,0.265204,0.097919,0.267678,0.425840,0.425526,0.021841,0.488074,0.448583,Buy
1,0.618853,0.098269,0.612803,0.388009,0.129931,0.419508,0.446633,0.632983,0.976337,0.506613,0.370071,Buy
2,0.616342,0.084727,0.623019,0.375479,0.127849,0.368689,0.407978,0.495000,0.572345,0.472149,0.410345,Buy
3,0.626805,0.144399,0.627725,0.252964,0.174034,0.458219,0.636463,0.008021,0.310170,0.675868,0.268573,Buy
4,0.630542,0.078427,0.633952,0.405530,0.149058,0.372005,0.378074,0.935303,0.095816,0.445485,0.422035,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...
1065,0.046331,0.015352,0.044543,0.283451,0.054807,0.152018,0.573371,0.598821,0.667242,0.614030,0.314889,Buy
1066,0.044734,0.013325,0.044985,0.396460,0.054509,0.278246,0.578553,0.505677,0.403398,0.624234,0.249431,Buy
1067,0.045238,0.009986,0.045535,0.438230,0.057879,0.235029,0.487527,0.295098,0.220715,0.528060,0.324231,Buy
1068,0.046563,0.009708,0.045326,0.354294,0.063132,0.190013,0.534080,0.340935,0.528174,0.517756,0.375548,Buy


In [ ]:
# make but keep and sell dfs

# buy df
buy_df = normalized_df_decisions[normalized_df_decisions['Buy/Sell/Keep'] == 'Buy']

sell_df = normalized_df_decisions[normalized_df_decisions['Buy/Sell/Keep'] == 'Sell']

keep_df = normalized_df_decisions[normalized_df_decisions['Buy/Sell/Keep'] == 'Keep']

buy_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
sell_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
keep_df.drop(columns=['Buy/Sell/Keep'], inplace=True)

/tmp/ipykernel_217425/4201824248.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
/tmp/ipykernel_217425/4201824248.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_df.drop(columns=['Buy/Sell/Keep'], inplace=True)
/tmp/ipykernel_217425/4201824248.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_df.drop(columns=['Buy/Sell/Keep'], inplace=True)


In [ ]:
keep_df.values[0]

array([0.68520936, 0.21044247, 0.70654029, 0.32968519, 0.40148662,
       0.3234407 , 0.41198171, 0.65831765, 0.35153954, 0.47571796,
       0.42923899])

In [ ]:
next_vectors

[{'next_day_vector': [1402.62,
   1459.251255724728,
   61.63517479419708,
   2.5,
   15.56,
   36.95797198,
   12.6084056,
   1499,
   50.43362241,
   36.95797198,
   24.19173179],
  'comp': 'AMZN'},
 {'next_day_vector': [90.56,
   92.9860921806097,
   3.5420436573028495,
   4.5,
   7.8,
   37.92667509,
   14.91782554,
   791,
   47.15549937,
   37.92667509,
   14.97491294],
  'comp': 'MSFT'},
 {'next_day_vector': [36.48,
   36.048149756491185,
   1.0358777984261485,
   2.5,
   17.26,
   21.22905028,
   8.938547486,
   179,
   69.83240223,
   26.09952618,
   8.938547486],
  'comp': 'PFIZER'}]

In [ ]:
# similarty 
from sklearn.metrics.pairwise import cosine_similarity

buy_score = []
sell_score = []
keep_score = []

for i in range(len(next_vectors)):
    buy_score_similarity = 0
    c = len(buy_df)
    buy_score_similarity = cosine_similarity(buy_df.values, [next_vectors[i]['next_day_vector']])
    total_cos_buy_score = np.sum(buy_score_similarity)
    buy_score_similarity/=c
    buy_score.append({'buyscore': total_cos_buy_score, 'comp': next_vectors[i]['comp']})
    
    
    sell_score_similarity = 0
    c = len(sell_df)

    sell_score_similarity = cosine_similarity(sell_df.values, [next_vectors[i]['next_day_vector']])
    sell_score_similarity/=c
    total_cos_sell_score = np.sum(sell_score_similarity)
    sell_score.append({'sellscore': total_cos_sell_score, 'comp': next_vectors[i]['comp']})
    
    keep_score_similarity = 0
    c = len(keep_df)
    
    keep_score_similarity = cosine_similarity(keep_df.values, [next_vectors[i]['next_day_vector']])
    keep_score_similarity/= c
    total_cos_keep_score = np.sum(keep_score_similarity)
    keep_score.append({'keepscore': total_cos_keep_score, 'comp': next_vectors[i]['comp']})



In [ ]:
company_scores = {}

# Merge the scores for each company
for score_list, score_type in zip([buy_score, sell_score, keep_score], ['buy', 'sell', 'keep']):
    for score in score_list:
        comp = score['comp']
        score_value = score[score_type + 'score']
        if comp not in company_scores:
            company_scores[comp] = {'buy': None, 'sell': None, 'keep': None}
        company_scores[comp][score_type] = score_value

print(company_scores)

{'AMZN': {'buy': 181.8513670810368, 'sell': 0.2778936113202055, 'keep': 0.2775164464104768}, 'MSFT': {'buy': 243.71618930373774, 'sell': 0.46432249582296203, 'keep': 0.4357477791964119}, 'PFIZER': {'buy': 322.02227601684234, 'sell': 0.6000482167727135, 'keep': 0.5870774486156017}}


In [ ]:
buy_basket = []
sell_basket = []
keep_basket = []

for company, scores in company_scores.items():
    buy_score = scores['buy']
    sell_score = scores['sell']
    keep_score = scores['keep']
    
    if buy_score is not None and (sell_score is None or buy_score > sell_score) and (keep_score is None or buy_score > keep_score):
        buy_basket.append(company)
    elif sell_score is not None and (buy_score is None or sell_score > buy_score) and (keep_score is None or sell_score > keep_score):
        sell_basket.append(company)
    elif keep_score is not None and (buy_score is None or keep_score > buy_score) and (sell_score is None or keep_score > sell_score):
        keep_basket.append(company)

print("Buy Basket:", buy_basket)
print("Sell Basket:", sell_basket)
print("Keep Basket:", keep_basket)



Buy Basket: ['AMZN', 'MSFT', 'PFIZER']
Sell Basket: []
Keep Basket: []


In [ ]:
top_n = 2

sorted_buy = sorted(buy_score, key=lambda x: x['buyscore'])
sorted_sell = sorted(sell_score, key=lambda x: x['sellscore'])
sorted_keep = sorted(keep_score, key=lambda x: x['keepscore'])

print(sorted_sell)
print(sorted_buy)
print(sorted_keep)

top_buy = sorted_buy[:top_n]
top_sell = sorted_sell[:top_n]
top_keep = sorted_keep[:top_n]


TypeError: 'numpy.float64' object is not iterable

In [ ]:
# Assuming sorted_buy, sorted_sell, and sorted_keep contain tuples of (score, company)
top_buy = sorted_buy[:top_n][::-1]

top_buy_stocks = set(stock for stock in top_buy)

filtered_sell = [sell_stock for sell_stock in sorted_sell if sell_stock not in top_buy_stocks]

filtered_sell = sorted(sell_score, key=lambda x: x['sellscore'])

top_sell_filtered = filtered_sell[:top_n][::-1]

top_sell = top_sell_filtered

top_sell_stocks = set(stock for stock in top_sell)

filtered_keep = [keep_stock for keep_stock in sorted_keep if keep_stock not in top_sell_stocks]

filtered_keep = sorted(keep_score, key=lambda x: x['keepscore'])
top_keep_filtered = filtered_keep[:top_n][::-1]
top_keep = top_keep_filtered



print(top_buy)
print(top_sell)
print(top_keep)


TypeError: unhashable type: 'dict'

In [ ]:
print('Stocks to buy: ')
for i in range(len(top_buy)):
    print(top_buy[i]['comp'])

Stocks to buy: 
PFIZER
AMZN


In [ ]:
print('Stocks to Sell')

for i in range(len(top_sell)):
    print(top_sell[i]['comp'])

Stocks to Sell
MSFT
AMZN


In [ ]:
print('Stocks to Keep')

for i in range(len(top_keep)):
    print(top_keep[i]['comp'])

Stocks to Keep
MSFT
AMZN
